In [1]:
# Corpus : 학습시킬 데이터
corpus = ["I love natural language processing", "Word2vec is a fascinating model", "Understanding embeddings is key"]

print(f"원본 Corpus : {corpus}")

원본 Corpus : ['I love natural language processing', 'Word2vec is a fascinating model', 'Understanding embeddings is key']


In [2]:
#토큰화
tokenized_corpus = []
for sentence in corpus:
    tokens = sentence.lower().split()
    tokenized_corpus.append(tokens)

print(f"토큰화된 corpus: {tokenized_corpus}")

토큰화된 corpus: [['i', 'love', 'natural', 'language', 'processing'], ['word2vec', 'is', 'a', 'fascinating', 'model'], ['understanding', 'embeddings', 'is', 'key']]


In [ ]:
# vocabulary set 만들기
vocabulary = set() # set은 중복 단어 허용 x인거 이용
for tokens in tokenized_corpus:
    vocabulary.update(tokens)

print(f"Vocabulary (고유 단어 집합) : {vocabulary}")

Vocabulary (고유 단어 집합) : {'key', 'love', 'language', 'i', 'natural', 'is', 'a', 'understanding', 'processing', 'embeddings', 'word2vec', 'model', 'fascinating'}


In [4]:
# integer id부여하기 : 두 종류의 딕셔너리 만들기
sorted_vocab = sorted(list(vocabulary))

word_to_id = {word: i for i, word in enumerate(sorted_vocab)}
id_to_word = {i: word for i,word in enumerate(sorted_vocab)}

print(f"Vocabulary(정렬) : {sorted_vocab}")
print(f"\n단어 -> ID 딕셔너리 : {word_to_id}")
print(f"\nID->단어 딕셔너리: {id_to_word}")

Vocabulary(정렬) : ['a', 'embeddings', 'fascinating', 'i', 'is', 'key', 'language', 'love', 'model', 'natural', 'processing', 'understanding', 'word2vec']

단어 -> ID 딕셔너리 : {'a': 0, 'embeddings': 1, 'fascinating': 2, 'i': 3, 'is': 4, 'key': 5, 'language': 6, 'love': 7, 'model': 8, 'natural': 9, 'processing': 10, 'understanding': 11, 'word2vec': 12}

ID->단어 딕셔너리: {0: 'a', 1: 'embeddings', 2: 'fascinating', 3: 'i', 4: 'is', 5: 'key', 6: 'language', 7: 'love', 8: 'model', 9: 'natural', 10: 'processing', 11: 'understanding', 12: 'word2vec'}


In [5]:
# CBow모델의 핵심은 context로 center예측
# 즉, (context, center) pair로 만들어줘야
# window 크기는 2로 설정

window_size = 2
cbow_pairs = []

for tokens in tokenized_corpus:
    for i, center_word in enumerate(tokens):
        context_words = []
        for j in range(i-window_size, i+window_size+1):
            if i==j or j<0 or j>= len(tokens):
                continue
            context_words.append(tokens[j])
        center_word_id = word_to_id[center_word]
        context_word_ids = [word_to_id[word] for word in context_words]

        cbow_pairs.append((context_word_ids,center_word_id))
print("CBOW 학습 데이터 쌍 (첫 5개만):")
for pair in cbow_pairs[:5]:
    context_words = [id_to_word[id_] for id_ in pair[0]]
    center_word = id_to_word[pair[1]]
    print(f"Context: {context_words} -> center: {center_word}")

CBOW 학습 데이터 쌍 (첫 5개만):
Context: ['love', 'natural'] -> center: i
Context: ['i', 'natural', 'language'] -> center: love
Context: ['i', 'love', 'language', 'processing'] -> center: natural
Context: ['love', 'natural', 'processing'] -> center: language
Context: ['natural', 'language'] -> center: processing


In [7]:
!pip install torch

  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.0
    Uninstalling typing-extensions-3.10.0.0:
      Successfully uninstalled typing-extensions-3.10.0.0


In [ ]:
#cbow using pytorch
#주변단어들의 임베딩 벡터들을 평균내서 -> score 계산 -> 중심단어의 점수와 가까워지도록
import torch
import torch.nn as nn

# 6. CBOW 모델 정의
# 파이토치의 nn.Module을 상속받아 모델 클래스 만들기
class CBOW(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        # super().__init__()는 nn.Module의 초기화 함수를 먼저 실행하라는 거
        super(CBOW, self).__init__()
        
        # 1. 임베딩 레이어 (Embedding Layer)
        # vocab_size: 전체 단어 개수
        # embedding_dim: 각 단어를 표현할 벡터의 차원(크기)
        # 이 레이어는 단어 ID를 받아서 해당 단어의 임베딩 벡터를 찾아줌
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        
        # 2. 선형 레이어 (Linear Layer)
        # 평균낸 임베딩 벡터를 입력으로 받아, 전체 단어에 대한 점수(logits)를 출력
        self.linear = nn.Linear(embedding_dim, vocab_size)

    # 모델이 데이터를 입력받았을 때 순서대로 실행되는 부분
    def forward(self, context_word_ids):
        # context_word_ids로 임베딩 벡터들을 조회함
        context_embs = self.embeddings(context_word_ids)
        
        # 벡터들의 평균을 계산, dim=0은 텐서의 첫번째 차원을 기준으로 평균을 내라는 거
        mean_embs = torch.mean(context_embs, dim=0)
        
        # 평균낸 벡터를 선형 레이어에 통과시켜 점수를 계산
        logits = self.linear(mean_embs)
        
        return logits

# 모델 학습에 필요한 파라미터(Parameter) 설정
VOCAB_SIZE = len(vocabulary)
EMBEDDING_DIM = 10 # 각 단어를 10차원 벡터로 표현

# 모델 객체 생성
model = CBOW(VOCAB_SIZE, EMBEDDING_DIM)
print(model)

CBOW(
  (embeddings): Embedding(13, 10)
  (linear): Linear(in_features=10, out_features=13, bias=True)
)


In [ ]:
# 7. 모델 학습
import torch.optim as optim

# 손실 함수와 옵티마이저(최적화 도구)를 정의함.
# CrossEntropyLoss는 분류 문제에서 주로 사용하는 손실 함수임
loss_function = nn.CrossEntropyLoss()
# Adam은 파라미터를 효율적으로 업데이트해주는 최적화 알고리즘 중 하나
# model.parameters()는 학습시킬 모든 파라미터(임베딩, 선형층의 가중치 등)를 의미합니다.
optimizer = optim.Adam(model.parameters(), lr=0.01)

print("학습 시작!")
# 학습 사이클을 100번 반복 (100 Epochs).
for epoch in range(100):
    total_loss = 0
    
    # 준비해둔 학습 데이터 쌍을 하나씩 가져와서 학습
    for context_ids, center_id in cbow_pairs:
        # Pytorch는 데이터를 Tensor라는 자료구조로 다룬다
        # context_ids를 Tensor로 변환
        context_t = torch.tensor(context_ids, dtype=torch.long)
        
        # 모델은 항상 미니배치(mini-batch) 단위로 입력을 받도록 설계되어 있으므로
        # view(1, -1)를 통해 [1, vocab_size] 형태의 텐서로 만들어준다
        target_t = torch.tensor([center_id], dtype=torch.long)
        
        # 이전 학습에서 계산된 기울기(gradient)를 모두 0으로 초기화
        model.zero_grad()
        
        # 1. 예측 (Forward Pass)
        # 모델에 주변 단어 ID 텐서를 입력하여 예측 점수(logits)를 받는다
        logits = model(context_t)

        # 2. 손실 계산 (Calculate Loss)
        # 예측 점수와 실제 정답 ID를 비교하여 손실을 계산함
        loss = loss_function(logits.view(1, -1), target_t)
        
        # 3. 업데이트 (Backward Pass & Optimization)
        # 손실을 기반으로 각 파라미터가 얼마나 변해야 하는지(기울기) 계산
        loss.backward()
        # 옵티마이저가 계산된 기울기를 바탕으로 모델의 파라미터를 업데이트
        optimizer.step()
        
        total_loss += loss.item()
    
    # 10번의 epoch마다 중간 손실 값을 출력
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/100 | Total Loss: {total_loss:.4f}")

print("학습 완료!")

학습 시작!
Epoch 10/100 | Total Loss: 16.6847
Epoch 20/100 | Total Loss: 7.3471
Epoch 30/100 | Total Loss: 3.9937
Epoch 40/100 | Total Loss: 2.7365
Epoch 50/100 | Total Loss: 2.2091
Epoch 60/100 | Total Loss: 1.9520
Epoch 70/100 | Total Loss: 1.8096
Epoch 80/100 | Total Loss: 1.7228
Epoch 90/100 | Total Loss: 1.6658
Epoch 100/100 | Total Loss: 1.6263
학습 완료!


In [ ]:
# 8. 학습 결과 확인: 특정 단어와 가장 유사한 단어 찾기
from scipy.spatial.distance import cosine

# 모델의 임베딩 파라미터를 가져오기
# .detach().numpy()는 학습 그래프에서 분리하여 numpy 배열로 변환하는 코드
embeddings = model.embeddings.weight.detach().numpy()

# 기준 단어 설정
target_word = 'language'

# 기준 단어의 ID와 임베딩 벡터를 가져온다
target_id = word_to_id[target_word]
target_vec = embeddings[target_id]

# 모든 단어와의 코사인 유사도를 저장할 딕셔너리
similarities = {}

# 모든 단어를 순회하며 기준 단어와의 유사도를 계산한다.
for word, i in word_to_id.items():
    if word == target_word:
        continue
    
    # 코사인 거리를 계산 (1 - 코사인 유사도)
    dist = cosine(target_vec, embeddings[i])
    # 코사인 유사도는 1 - 거리
    similarity = 1 - dist
    similarities[word] = similarity

# 유사도가 높은 순서대로 단어를 정렬.
# key=lambda item: item[1]는 딕셔너리의 값(value)을 기준으로 정렬하라는 의미
# reverse=True는 내림차순(큰 것부터)으로 정렬
sorted_similar_words = sorted(similarities.items(), key=lambda item: item[1], reverse=True)

print(f"'{target_word}'와(과) 가장 유사한 단어 Top 5:")
for word, sim in sorted_similar_words[:5]:
    print(f"{word}: {sim:.4f}")

'language'와(과) 가장 유사한 단어 Top 5:
model: 0.2510
processing: 0.2300
natural: 0.1447
i: 0.0701
love: 0.0039
